### Importar librerias

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
from flask import Flask, request, jsonify

### Cargar el dataset

In [3]:
# Cargar el dataset
data = pd.read_csv('..\API\data_final_ML.csv')

### Modelo

In [4]:
# Preprocesamiento de datos
# Crear una matriz TF-IDF para el texto del título de las películas
stopwords_custom = ["the", "and", "in", "of"]  # Stopwords personalizados
tfidf = TfidfVectorizer(stop_words=stopwords_custom)
tfidf_matrix = tfidf.fit_transform(data['title'])

# Calcular la similitud del coseno entre los títulos de las películas
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Crear un índice para los títulos
indices = pd.Series(data.index, index=data['title']).drop_duplicates()

# Función de recomendación


def recomendacion(titulo):
    # Verificar si el título está en el DataFrame
    if titulo not in indices:
        return {"error": f"No se encontró ninguna película con el título '{titulo}'."}

    # Encontrar el índice de la película con el título dado
    idx = indices[titulo]

    # Calcular las puntuaciones de similitud de todas las películas con la película dada
    sim_scores = list(enumerate(cosine_similarities[idx]))

    # Ordenar las películas por puntaje de similitud en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las películas más similares (excluyendo la película dada)
    sim_scores = sim_scores[1:6]  # Obtener las 5 películas más similares
    movie_indices = [x[0] for x in sim_scores]

    # Devolver los títulos de las películas más similares
    respuesta_recomendacion = data['title'].iloc[movie_indices].tolist()
    return {'lista recomendada': respuesta_recomendacion}

### Prueba

In [5]:
recomendacion("Paranormal Activity: The Ghost Dimension")

{'lista recomendada': ['Paranormal Activity 3',
  'Paranormal Activity 4',
  'Paranormal Activity: The Marked Ones',
  'Ghost',
  'Ghost World']}